In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from nltk.tokenize import word_tokenize
import re

## Read all corpora

In [2]:
# Expressão regular para encontrar os padrões verbo-id
padrao = r'\b(\w+)-\d{2}\b'

# Função de substituição
def substituir(match):
    return match.group(1)

In [3]:
###################
# NEWS
###################
path_news = '../AMR-BP/AMRNews/unsplit/amr.txt' 

with open(path_news,'r') as file:
    
    news_corpus = file.read()
    
###################
# opisums
###################
path_opisums = '../AMR-BP/OpiSums-PT-AMR/amr-opisums-pt.txt'

with open(path_opisums,'r') as file:
    
    opisums_corpus = file.read()

###################
# Little Prince
###################
path_lp = '../AMR-BP/AMR-LittlePrince/amr_br-v1.0.xml'

with open(path_lp,'r') as file:
    
    lp_corpus = file.read()
    
###################
# AMRScien
###################
path_sci = '../AMRScien-Br-Corpus/AMRScien-Br.txt'

with open(path_sci,'r') as file:
    
    sci_corpus = file.read()

## Join corpora

In [4]:
str_final = ""

### News

In [5]:
def has_alignments_bramr(amr_sentence):
    # Percorre as linhas da sentença AMR
    for line in amr_sentence.split('\n'):
        # Verifica se a linha começa com '# ::alignments-bramr'
        if line.startswith('# ::alignments-bramr'):
            # Verifica se a linha contém algo além de '::alignments-bramr'
            if line.strip() != '# ::alignments-bramr':
                return True
            else:
                return False
    # Retorna False se o marcador não for encontrado
    return False

In [6]:
print(news_corpus)



# ::id poder-2-35-32
# ::tok-pt Tem que voltar já .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 1-1 2-1.1 0-1.1.1 3-1.1.2
(o / obligate-01~e.1 :ARG2 (v / voltar-01~e.2 :ARG1 (e / ele~e.0) :time (j / já~e.3)))

# ::id poder-2-35-16
# ::tok-pt Não se trata de liberdade ?
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 2-1 5-1.1 4-1.2
(t / tratar-01~e.2 :ARG1 (r / request-confirmation-91~e.5) :ARG2 (l / liberdade~e.4))

# ::id poder-2-35-18
# ::tok-pt Até me ameaçaram de morte .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 2-1 4-1.1 1-1.2
(a / ameaçar-01~e.2 :ARG1 (m / morrer-01~e.4 :ARG1 e) :ARG2 (e / eu~e.1))

# ::id poder-2-35-19
# ::tok-pt Sei todas as dificuldades .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 0-1 3-1.2 1-1.2.1.r 1-1.2.1
(s / saber-01~e.0 :ARG0 (e / eu) :ARG1 (d / dificuldade~e.3 :all~e.1 (t / todo~e.1)))

# ::id poder-2-86-31
# ::tok-pt Mas não com essa dimensão .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 0-1.1.r 4-1.1

In [7]:
news_corpus = news_corpus.replace("tok-pt", "snt")

In [8]:
blocos = news_corpus.split("\n\n")
blocos = [bloco for bloco in blocos if bloco != ""]
blocos

['# ::id poder-2-35-32\n# ::snt Tem que voltar já .\n# ::tok-en \n# ::alignments-bren \n# ::alignments-bramr 1-1 2-1.1 0-1.1.1 3-1.1.2\n(o / obligate-01~e.1 :ARG2 (v / voltar-01~e.2 :ARG1 (e / ele~e.0) :time (j / já~e.3)))',
 '# ::id poder-2-35-16\n# ::snt Não se trata de liberdade ?\n# ::tok-en \n# ::alignments-bren \n# ::alignments-bramr 2-1 5-1.1 4-1.2\n(t / tratar-01~e.2 :ARG1 (r / request-confirmation-91~e.5) :ARG2 (l / liberdade~e.4))',
 '# ::id poder-2-35-18\n# ::snt Até me ameaçaram de morte .\n# ::tok-en \n# ::alignments-bren \n# ::alignments-bramr 2-1 4-1.1 1-1.2\n(a / ameaçar-01~e.2 :ARG1 (m / morrer-01~e.4 :ARG1 e) :ARG2 (e / eu~e.1))',
 '# ::id poder-2-35-19\n# ::snt Sei todas as dificuldades .\n# ::tok-en \n# ::alignments-bren \n# ::alignments-bramr 0-1 3-1.2 1-1.2.1.r 1-1.2.1\n(s / saber-01~e.0 :ARG0 (e / eu) :ARG1 (d / dificuldade~e.3 :all~e.1 (t / todo~e.1)))',
 '# ::id poder-2-86-31\n# ::snt Mas não com essa dimensão .\n# ::tok-en \n# ::alignments-bren \n# ::alignment

In [9]:
list_news_with_align = []
list_news_without_align = []
for bloco in blocos:
    
    if has_alignments_bramr(bloco):
        list_news_with_align.append(bloco)
    else:
        list_news_without_align.append(bloco)
        
print(len(list_news_without_align))
print(len(list_news_with_align))

244
626


In [10]:
news_without_align = "\n\n".join(list_news_without_align)
news_with_align = "\n\n".join(list_news_with_align)

In [11]:
str_final += news_corpus

### opisums

In [12]:
print(opisums_corpus)

# ::id Fala-Serio-Pai.Documento_0.0
# ::snt Bom .
(b / bom)

# ::id Fala-Serio-Mae.Documento_8.3
# ::snt Leia !
(l / ler-01 :mode imperative)

# ::id Fala-Serio-Amiga.Documento_18.0
# ::snt Amei !
(a / amar-01
      :ARG0 (e / eu))

# ::id Iphone-5.Documento_6.0
# ::snt Perfeito
(p / perfeito)

# ::id Iphone-5.Documento_4.0
# ::snt Iphone 5
(p / product :name (n / name :op1 "Iphone" :op2 "5"))

# ::id Fala-Serio-Amiga.Documento_27.0
# ::snt Muito bom
(b / bom
      :mod (m / muito))

# ::id O-Reverso-da-Medalha.Documento_47.0
# ::snt Diamantes
(d / diamante)

# ::id Iphone-5.Documento_2.0
# ::snt Excelente
(e / excelente)

# ::id O-Reverso-da-Medalha.Documento_49.0
# ::snt Perfeito .
(p / perfeito)

# ::id O-Apanhador-no-Campo-de-Centeio.Documento_98.0
# ::snt Perfeito !
(p / perfeito)

# ::id Fala-Serio-Amiga.Documento_0.5
# ::snt ( poucas )
(p / pouco)

# ::id Iphone-5.Documento_0.0
# ::snt Este é dez
(s / score-on-scale-91
      :ARG1 10
      :domain (e / este))

# ::id Fala-Serio-

In [13]:
str_final += opisums_corpus

In [14]:
print(str_final)



# ::id poder-2-35-32
# ::snt Tem que voltar já .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 1-1 2-1.1 0-1.1.1 3-1.1.2
(o / obligate-01~e.1 :ARG2 (v / voltar-01~e.2 :ARG1 (e / ele~e.0) :time (j / já~e.3)))

# ::id poder-2-35-16
# ::snt Não se trata de liberdade ?
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 2-1 5-1.1 4-1.2
(t / tratar-01~e.2 :ARG1 (r / request-confirmation-91~e.5) :ARG2 (l / liberdade~e.4))

# ::id poder-2-35-18
# ::snt Até me ameaçaram de morte .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 2-1 4-1.1 1-1.2
(a / ameaçar-01~e.2 :ARG1 (m / morrer-01~e.4 :ARG1 e) :ARG2 (e / eu~e.1))

# ::id poder-2-35-19
# ::snt Sei todas as dificuldades .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 0-1 3-1.2 1-1.2.1.r 1-1.2.1
(s / saber-01~e.0 :ARG0 (e / eu) :ARG1 (d / dificuldade~e.3 :all~e.1 (t / todo~e.1)))

# ::id poder-2-86-31
# ::snt Mas não com essa dimensão .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 0-1.1.r 4-1.1 1-1.1.1.r 1-1.

### Little prince

!!! PRECISA FAZER ALGUMAS MUDANCAS NO ARQUIVO DE ORIGEM !!!



LEMBRANDO QUE HÁ UM ERRO NA SENTENÇA 43 e 68 (falta fechamento)



O parser nao entende o 1 em:
	<sntamr>
		<sentence id="1318">E a cor de mel me fazia feliz.</sentence>
		<amr date="2017-22-10T13:10">
			(e / e
				:op2 (f / fazer-11
					:ARG0 (c / cor
						:mod (m / mel))
					:ARG1 (e1 / eu)
					:ARG2 (1 / feliz)))
		</amr>
		
	</sntamr>


Nesse caso, foi retirado a sentença
    

In [15]:
print(lp_corpus)

<!DOCTYPE amrset SYSTEM "amrset.dtd">
<amrset name="le-petit-prince" version="1.0" date="2017-02-16" corpus="O Pequeno Príncipe, por Antoine Saint-Exupery">
	<sntamr>
		<sentence id="1">Capítulo 1.</sentence>
		<amr date="2017-02-16T10:22"> 
			(c / capítulo :mod 1) 
		</amr>
	</sntamr>
	<sntamr>
		<sentence id="2">Certa vez, quando tinha seis anos, vi num livro sobre a Floresta Virgem, "Histórias Vividas", uma imponente gravura.</sentence>
		<amr date="2017-02-16T10:22"> 
			(v / ver-01 
				:ARG0 (e / eu) 
				:ARG1 (g / gravura 
					:mod (i / imponente) 
					:location (l / livro 
						:name (n / name :op1 "Histórias" :op2 "Vividas") 
						:topic (f / floresta 
							:mod (v1 /virgem)))) 
				:mod (c / certa-vez) 
				:time (i1 / idade 
					:ARG1 e 
					:ARG2 (t / temporal-quantity 
						:quant 6 :unit (a / ano))))
		</amr>
	</sntamr>
	<sntamr>
		<sentence id="3">Representava ela uma jibóia que engolia uma fera.</sentence>
		<amr date="2017-02-16T10:43">
			(r / representar-0

In [16]:
lp_str = ""

In [17]:
# Carrega e faz o parsing do arquivo XML
tree = ET.parse(path_lp)
root = tree.getroot()

In [18]:
# Nome do corpus
corpus_name = root.attrib.get('name')
lp_str = ""


for sntamr in root.findall('sntamr'):
    sentence_id = sntamr.find('sentence').attrib['id'].replace(';', '-')
    sentence_text = sntamr.find('sentence').text
    sentence_text_tk = " ".join(word_tokenize(sentence_text))
    amr_text = sntamr.find('amr').text.strip()
    
    lp_str+=f"# ::id {corpus_name}-{sentence_id}\n"
    lp_str+=f"# ::snt {sentence_text_tk}\n"
    lp_str+=f"{amr_text}\n\n"

In [19]:
print(lp_str)

# ::id le-petit-prince-1
# ::snt Capítulo 1 .
(c / capítulo :mod 1)

# ::id le-petit-prince-2
# ::snt Certa vez , quando tinha seis anos , vi num livro sobre a Floresta Virgem , `` Histórias Vividas '' , uma imponente gravura .
(v / ver-01 
				:ARG0 (e / eu) 
				:ARG1 (g / gravura 
					:mod (i / imponente) 
					:location (l / livro 
						:name (n / name :op1 "Histórias" :op2 "Vividas") 
						:topic (f / floresta 
							:mod (v1 /virgem)))) 
				:mod (c / certa-vez) 
				:time (i1 / idade 
					:ARG1 e 
					:ARG2 (t / temporal-quantity 
						:quant 6 :unit (a / ano))))

# ::id le-petit-prince-3
# ::snt Representava ela uma jibóia que engolia uma fera .
(r / representar-01 
				:domain (e / ela) 
				:topic (j / jibóia 
					:ARG0-of (e1 / engolir-01 
						:ARG1 (f / fera))))

# ::id le-petit-prince-4
# ::snt Eis a cópia do desenho .
(b / be-located-at-91 
				:ARG2 (d / desenho
					:mod (c / cópia)))

# ::id le-petit-prince-5
# ::snt Dizia o livro : `` As jibóias engolem , se

In [20]:
# Substituição utilizando re.sub com a função de substituição
#lp_str = re.sub(padrao, substituir, lp_str)

In [21]:


str_final += lp_str

In [22]:
print(str_final)



# ::id poder-2-35-32
# ::snt Tem que voltar já .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 1-1 2-1.1 0-1.1.1 3-1.1.2
(o / obligate-01~e.1 :ARG2 (v / voltar-01~e.2 :ARG1 (e / ele~e.0) :time (j / já~e.3)))

# ::id poder-2-35-16
# ::snt Não se trata de liberdade ?
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 2-1 5-1.1 4-1.2
(t / tratar-01~e.2 :ARG1 (r / request-confirmation-91~e.5) :ARG2 (l / liberdade~e.4))

# ::id poder-2-35-18
# ::snt Até me ameaçaram de morte .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 2-1 4-1.1 1-1.2
(a / ameaçar-01~e.2 :ARG1 (m / morrer-01~e.4 :ARG1 e) :ARG2 (e / eu~e.1))

# ::id poder-2-35-19
# ::snt Sei todas as dificuldades .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 0-1 3-1.2 1-1.2.1.r 1-1.2.1
(s / saber-01~e.0 :ARG0 (e / eu) :ARG1 (d / dificuldade~e.3 :all~e.1 (t / todo~e.1)))

# ::id poder-2-86-31
# ::snt Mas não com essa dimensão .
# ::tok-en 
# ::alignments-bren 
# ::alignments-bramr 0-1.1.r 4-1.1 1-1.1.1.r 1-1.

### Sci

As sentenças de id 66, 85, 125 e 169 dão erro no parser por conta dos "anos". (precisa entender o motivo)

182 precisa fechar ( no começo da frase

In [23]:
print(sci_corpus)

# ::id 1
# ::snt Meyer tem duas explicações para esses resultados. 
(h3 / ter-01
	:ARG0 (p0 / person
		:name (n1 / name
			:op1 "Meyer")
	:ARG1 (e5 / explicar-01
		:ARG0 p0
		:ARG1 (t8 / coisa
			:mod (t7 / esse)			
			:ARG2-of (r9 / resultar-01))
		:quant 2)
	:wiki -))
	
# ::id 2
# ::snt Essa programação pode ser alterada quando necessário. 
(p2 / possible-01
	:ARG1 (a3 / alterar-01
		:ARG1 (p1 / programar-01
			:mod (t0 / esse)
		:time (n4 /necessário))))	

# ::id 3
# ::snt Novo modelo de desenvolvimento para São Paulo
(m2 / modelo
	:topic (d3 / desenvolver-01
		:ARG1 (l4 / location
			:name (n5 / name
				:op2 "Paulo"
				:op1 "São")
			:wiki -))
	:mod (n / novo))

# ::id 4
# ::snt Isso poderia ser feito para a universidade. 
(p1 / possible-01
	:ARG1 (d2 / fazer-01
		:ARG1 (t0 / isso)
		:ARG3 (u3 / universidade)))

# ::id 5
# ::snt "No Brasil, essa técnica é pouco conhecida. 
(k6 / saber-01
	:location (c0 / country
		:name (n1 / name
			:op1 "Brasil")
		:wiki "Brazil")
	:ARG1 (t4 / 

In [24]:
str_final +=sci_corpus

## Save

In [25]:
with open('../data/raw/joined_corpora.txt', 'w') as file:
    
    file.write(str_final)

In [26]:
with open('../data/raw/4aligner_little_prince.txt', 'w') as file:
    
    file.write(lp_str)
    
with open('../data/raw/4aligner_opisums.txt', 'w') as file:
    
    file.write(opisums_corpus)
    
with open('../data/raw/4aligner_news.txt', 'w') as file:
    
    file.write(news_corpus)
    
with open('../data/raw/4aligner_news_without_alignment.txt', 'w') as file:
    
    file.write(news_without_align)
    
with open('../data/raw/4aligner_news_with_alignment.txt', 'w') as file:
    
    file.write(news_with_align)  

    
with open('../data/raw/4aligner_science.txt', 'w') as file:
    
    file.write(sci_corpus)